<a href="https://colab.research.google.com/github/officialdivyanshu/Document-Q-A-AI-Agent/blob/main/Document_Q%26A_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install pdfplumber faiss-cpu google-generativeai langchain langchain-text-splitters

In [31]:
from google.colab import userdata

# Get the API key from Colab Secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [32]:
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

In [8]:
from google.colab import files

uploaded_files = files.upload()
print("Uploaded files:", uploaded_files.keys())


Saving wearable finger oulse oximetre for continuos oxygen measurement.pdf to wearable finger oulse oximetre for continuos oxygen measurement.pdf
Uploaded files: dict_keys(['wearable finger oulse oximetre for continuos oxygen measurement.pdf'])


In [9]:
import pdfplumber

documents = []

for filename in uploaded_files.keys():
    with pdfplumber.open(filename) as pdf:
        full_text = ""
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                full_text += text + "\n"
    documents.append({
        "source": filename,
        "text": full_text
    })

print("Sample extracted text:\n")
print(documents[0]["text"][:1000])


Sample extracted text:

JMIR MHEALTH AND UHEALTH Buekers et al
Original Paper
Wearable Finger Pulse Oximetry for Continuous Oxygen
Saturation Measurements During Daily Home Routines of Patients
With Chronic Obstructive Pulmonary Disease (COPD) Over One
Week: Observational Study
Joren Buekers1,2, MSc; Jan Theunis1, MSc; Patrick De Boever1,3, PhD; Anouk W Vaes4, PhD; Maud Koopman4, MSc;
Eefje VM Janssen4, MSc; Emiel FM Wouters4,5, PhD; Martijn A Spruit4,5,6,7, PhD; Jean-Marie Aerts2, PhD
1Health Unit, Flemish Institute for Technological Research (VITO), Mol, Belgium
2Measure, Model & Manage Bioresponses, Department of Biosystems, KU Leuven, Leuven, Belgium
3Centre for Environmental Sciences, Hasselt University, Diepenbeek, Belgium
4Department of Research and Education, Centre of Expertise for Chronic Organ Failure (CIRO), Horn, Netherlands
5Department of Respiratory Medicine, Maastricht University Medical Centre, Maastricht, Netherlands
6Rehabilitation Research Center (REVAL), Biomedical

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

chunks = []
for doc in documents:
    parts = splitter.split_text(doc["text"])
    for part in parts:
        chunks.append({
            "text": part,
            "source": doc["source"]
        })

print("Total chunks created:", len(chunks))

Total chunks created: 77


In [11]:
pip install sentence-transformers


In [12]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load lightweight embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings for all chunks
texts = [chunk["text"] for chunk in chunks]
embeddings = embed_model.encode(texts, show_progress_bar=True)

print("✅ Local embeddings created with shape:", embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Local embeddings created with shape: (77, 384)


In [13]:
import faiss

embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

index.add(embeddings.astype("float32"))

print("✅ Total vectors stored in FAISS:", index.ntotal)


✅ Total vectors stored in FAISS: 77


In [14]:
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = embed_model.encode([query])[0]
    D, I = index.search(np.array([query_embedding]).astype("float32"), top_k)
    return [chunks[i]["text"] for i in I[0]]


In [15]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def ask_question(query):
    context_chunks = retrieve_relevant_chunks(query, top_k=3)
    context_text = "\n\n".join(context_chunks)

    prompt = f"""
You are an AI assistant. Answer the user's question strictly using the context below.

Context:
{context_text}

Question:
{query}

Give a clear and concise answer.
"""

    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)

    return response.text


In [16]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

models = genai.list_models()
for m in models:
    print(m.name)


models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025
models/embedding-001
models/text-embedding-004
models/gemini-embedding-001
models/aqa
models/

In [17]:
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = embed_model.encode([query])[0]
    D, I = index.search(np.array([query_embedding]).astype("float32"), top_k)
    return [chunks[i]["text"] for i in I[0]]

def ask_question(query):
    context_chunks = retrieve_relevant_chunks(query, top_k=3)
    context_text = "\n\n".join(context_chunks)

    prompt = f"""
You are an AI assistant. Answer the user's question strictly using the context below.

Context:
{context_text}

Question:
{query}

Give a clear and concise answer.
"""

    model = genai.GenerativeModel("models/gemini-flash-latest")
    response = model.generate_content(prompt)

    return response.text

In [33]:
while True:
    user_query = input("\nAsk a question (type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    answer = ask_question(user_query)
    print("\n🤖 AI Answer:\n", answer)



Ask a question (type 'exit' to quit): what is the coclusion of the report

🤖 AI Answer:
 Continuous SpO₂ measurements with wearable pulse oximeters identified significant SpO₂ fluctuations between and within multiple days and nights for the included patients.

Ask a question (type 'exit' to quit): exit
